<table>
<tr style="background-color:#FFFFFF;">
<td width=15%>
    <table>
        <tr><img src="./img/logo_uga.jpeg"></br></a>
        </tr>
        <tr><img src="./img/logo_uw.png"></br></a>
        </tr> 
    </table>
</td>
<td><center><h1>Supyquantile : A Toolbox for Minimization of Superquantile-Based Risk Measures</h1></center></td>
<td width=15%>
    <table>
        <tr><a href="https://yassine-laguel.github.io" style="font-size: 16px; font-weight: bold">Yassine Laguel </br></a>
        </tr> 
        <tr><a href="https://ljk.imag.fr/membres/Jerome.Malick/" style="font-size: 16px; font-weight: bold">Jerôme Malick </br></a>
        </tr>
        <tr><a href="http://faculty.washington.edu/zaid/" style="font-size: 16px; font-weight: bold">Zaid Harchaoui </br></a>
        </tr>    
    </table>
</td>
</tr>
</table>

<br/><br/><div id="top"></div>

<center><a style="font-size: 30pt; font-weight: bold">The Toolbox</a></center>

<br/>

<br/><br/><div id="part1_1"></div>
<a style="font-size: 20pt; font-weight: bold">A Generic Problem</a>
<br/>

Supyquantile is a python toolbox which is aimed at providing optimization first order convex algorithms for the minimization of superquantile based measures. In practice, it proposes to solve problems of the form :
                $$\min_{w \in \mathbb{R}^n}  Cvar(\hat{L}(w))$$
where $\hat{L} : w \mapsto (L(w,x_i,y_i))_{(1 \leq i \leq n)}$ takes an input $w \in \mathbb{R}^d$ and maps it to a discrete random variable composed of n outcomes.

Here the couples $(x_i, y_i) \in \mathbb{R}^d \times \mathbb{R}$ refer to the data provided by the user and associated to the problem. The function $L :\mathbb{R}^d \times \mathbb{R}^d \times \mathbb{R} \rightarrow \mathbb{R} $, also provided by the user is assumed to satisfy at least :
<ul>
      <li>$L$ is convex with respect to $w$</li>  
      <li>$L$ is subdifferentiable with respect to $w$</li>
</ul>


We show below two instances of this problem, <a href="#quantile_regression"> Quantile Regression </a> and <a href="#safe_least_squares"> Distributionally Robust Least Squares </a> and how our toolbox can treat them.

<br/><br/><div id="part1_2"></div>
<a style="font-size: 15pt; font-weight: bold">Functionning</a>
<br/>

The user can import toolbox's solver with statement : 

In [1]:
import os
import sys

sys.path.insert(0, os.path.abspath('..'))

from spqr.risk_optimization import RiskOptimizer

To create an instance of this solver, all the user needs is to provide a function $L$ as well as well as its subgradient (or gradient). 

In [2]:
import numpy as np

def L(w,x,y):
    return np.absolute(y-np.dot(x,w))
    
def L_prime(w,x,y):
    return np.sign(y-np.dot(x,w))

optimizer = RiskOptimizer(L,L_prime)  

Supyquantile inherits from scikit-learn estimator's class. Thus, it provides a <b>fit()</b> and a <b>predict()</b> that work as for usual scikit-learn estimators:

In [4]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

size = 500
dim = 2
# generate regression dataset
X, y = make_regression(n_samples=size, n_features=dim, noise=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


optimizer.fit(X_train,y_train)

y_pred = optimizer.predict(X_test)
print('Test Loss : ' + str(1.0/size *np.linalg.norm(y_pred - y_test)**2))

Test Loss : 5204948.38377226


Iterates of the algorithm and solution after fitting are available through :

In [5]:
list_iterates = optimizer.list_iterates
solution = optimizer.solution
print('Solution found :' + str(solution))

Solution found :[-2695.86172152 -2695.86172152]


The function $Cvar\circ L$ or its gradient at a given point $w$, one can use :

In [6]:
w = solution
value = optimizer.oracle.f(w,X_train,y_train)
gradient = optimizer.oracle.g(w,X_train,y_train)

print('Value found : ' + str(value))
print('Gradient found : ' + str(gradient))

Value found : 7071.868335287736
Gradient found : 0.10447761194029853


<br/><br/><div id="part1_2"></div>
<a style="font-size: 15pt; font-weight: bold">Custom Parameterization</a>
<br/>

The class <b>RiskOptimizer</b> can be parametrized in two different ways.

First option is to directly precise specific parameters wanted at instanciation of the class :

In [7]:
optimizer = RiskOptimizer(L,L_prime,p=0.72)  

Customizable parameters are :
<ul>
    <li><b>p</b>: the probability level chosen for Cvar (by default=0.8)</li>
    <li><b>algorithm</b>: the chosen first order method (by default='subgradient')</li>
    <li><b>w_start</b>: the starting point chosen for the iterative method (by default=$0_{\mathbb{R}^d}$)</li>
    <li><b>alpha</b>: constant involved in the learning rate adopted</li>
    <li><b>nb_iterations</b>: number of iterations wished for the algorithm</li>
</ul>

Second option is to provide a dictionary containing the specified parameters 


In [8]:
params = {
    # General Parameters
    'algorithm': 'catalyst',
    'w_start': None,
    'p': 0.8,
    'alpha': None,
    # Catalyst Parameters
    'catalyst_nb_iterations': 100,
    'catalyst_kappa': 100
    }

optimizer = RiskOptimizer(L,L_prime,params=params)  

<br/><br/><div id="part2"></div>
<a style="font-size: 20pt; font-weight: bold">Application Example</a>
<br/>




<br/><br/><div id="part3"></div>

<center><a style="font-size: 15pt; font-weight: bold"; id="#safe_least_squares"> Regularized Superquantile Regression </a></center>

<br/>

<p style="text-align: right; font-size: 10px;"><a href="#top">Go to top</a></p>

With Superquantile Robust Linear Regression, we try to solve the problem 
                         $$\min_{w \in \mathbb{R}^n}  Cvar(||Y - w^T X||^2) + \frac{\lambda}{2} ||w||^2$$
                         
With this model, we are trying to minimize the highest quantiles of the loss $||Y - w^T X||^2$, seen as a random variable depending on both $X$ and $Y$.

Let us first build some testing set for that purpose : 

In [10]:
from sklearn.preprocessing import scale 

dim = 2
size = 150
X, y = make_regression(n_samples=size, n_features=dim, noise=100.0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

def preprocess_dataset(x_set):
    x_set = scale(x_set)
    x_with_bias = np.ones((x_set.shape[0], x_set.shape[1] + 1))
    x_with_bias[:,:-1] = x_set
    return x_with_bias

X_train = preprocess_dataset(X_train)
X_test = preprocess_dataset(X_test)

We can build now our model :

In [11]:
lmbda = 1.0
def L(w,x,y):
    return (y-np.dot(x,w))**2 + lmbda/2.0 * np.dot(w[:-1],w[:-1])
def L_prime(w,x,y):
    w2 = np.copy(w)
    w2[-1] = 0
    return -2*(y-np.dot(x,w))*(x) + lmbda * w2 

regressor_safe = RiskOptimizer(L,L_prime, algorithm='bfgs', p=0.5)

We fit it :

In [12]:
regressor_safe.fit(X_train,y_train)

and observe the associated score on a testing set :

In [13]:
y_pred = regressor_safe.predict(X_test) 
print('Loss Robust Linear Prediction ' + str(1.0/size *np.linalg.norm(y_pred - y_test)**2))

Loss Robust Linear Prediction 2333.11931436067


A quick comparison with usual least squares gives :

In [14]:
from sklearn.linear_model import Ridge

ridge_regressor_l_2 = Ridge(alpha=lmbda/2.0,fit_intercept=False)
ridge_regressor_l_2.fit(X_train, y_train)
ground_y_pred = ridge_regressor_l_2.predict(X_test) 
print('Score for Sklearn Ridge Regression ' + str(1.0/size *np.linalg.norm(ground_y_pred - y_test)**2))

Score for Sklearn Ridge Regression 2862.1844456771587
